In [1]:
import torch
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base')
model = AutoModel.from_pretrained('facebook/dinov2-base')
model.eval()
inputs = processor(images=image, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
last_hidden_states = outputs[0]
print(inputs["pixel_values"].shape)
print(last_hidden_states.shape)

# We have to force return_dict=False for tracing
model.config.return_dict = False

with torch.no_grad():
    traced_model = torch.jit.trace(model, [inputs.pixel_values])
    traced_outputs = traced_model(inputs.pixel_values)
print("==============================================")
print((last_hidden_states - traced_outputs[0]).abs().max())
print("==============================================")

/home/lsaland/micromamba/envs/clip/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch.Size([1, 3, 224, 224])
torch.Size([1, 257, 768])


/home/lsaland/micromamba/envs/clip/lib/python3.11/site-packages/transformers/modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
/home/lsaland/micromamba/envs/clip/lib/python3.11/site-packages/transformers/models/dinov2/modeling_dinov2.py:158: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if num_channels != self.num_channels:
/home/lsaland/micromamba/envs/clip/lib/python3.11/site-packages/transformers/models/dinov2/modeling_dinov2.py:88: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the futu

tensor(3.4332e-05)


/home/lsaland/micromamba/envs/clip/lib/python3.11/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 282 / 197376 (0.1%)
Greatest absolute difference: 3.3795833587646484e-05 at index (0, 19, 137) (up to 1e-05 allowed)
Greatest relative difference: 0.0413491966450623 at index (0, 6, 422) (up to 1e-05 allowed)
  _check_trace(
/home/lsaland/micromamba/envs/clip/lib/python3.11/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 2. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2 / 768 (0.3%)
Greatest absolute difference: 1.6808509826660156e-05 at index (0, 282) (up to 1e-05 allowed)
Greatest relative difference: 0.0005818077592264318 at index (0, 185) (up to 1e-05 allowed)
  _check_trace(


In [ ]:
images = [Image.open("../../img/cat.png"),Image.open("../../img/cat.jpg")]
inputs = processor(images=images,return_tensors="pt")
outputs = model(**inputs,output_hidden_states=False)
print(outputs[0].shape)
print(outputs[1].shape)

In [12]:
import os
from tqdm import tqdm
data_path = "/data3/AID/Flickr2048/"
imgs = []
for file in tqdm(os.listdir(data_path)[:00]):
    imgs.append(Image.open(data_path + file))
inputs  = processor(images=imgs,return_tensors="pt")
outputs = model(**inputs)
print(outputs[1].shape)

100%|██████████| 100/100 [00:05<00:00, 17.08it/s]


KeyboardInterrupt: 

In [ ]:
outputs[0][1:,:].shape

In [ ]:
torch.mean(torch.eq(outputs[0][:,0,:],outputs[1]).float())

outputs[0][:,0,:] and outputs[1] both represent the class token